# Cherry Classifier Experiments Runner

This notebook runs a batch of experiments to optimize the cherry pit classifier.

**Experiments:**
1. **ResNet50 Augmented**: Baseline + Rotation/Affine/Jitter Augmentations.
2. **ResNet50 Scheduler**: Augmented + Cosine Annealing Learning Rate Scheduler.
3. **MobileNetV3**: Lightweight architecture with Augmentation + Scheduler.
4. **EfficientNet-B0**: Efficient architecture with Augmentation + Scheduler.

## Prerequisites
- Google Colab Pro (recommended for speed)
- Google Drive (for saving results)


## Step 1: Setup Environment

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 2. Install Dependencies
!pip install -q pyyaml scikit-learn matplotlib

# 3. Clone Repository (Training Code)
# REPLACE WITH YOUR REPO URL IF DIFFERENT
!git clone https://github.com/usefulmove/cherries.git

# 4. Clone Dataset (Shallow clone)
!git clone --depth 1 https://github.com/weshavener/cherry_classification.git

print("\nSetup complete!")

## Step 2: Run Experiments Loop
This cell will iterate through the defined configurations and run training for each. Results are saved to Google Drive.

In [ ]:
import subprocess
import os
from pathlib import Path

# Define experiments to run (paths relative to repo root)
configs = [
    "cherries/training/configs/resnet50_augmented.yaml",
    "cherries/training/configs/resnet50_scheduler.yaml",
    "cherries/training/configs/mobilenetv3_large.yaml",
    "cherries/training/configs/efficientnet_b0.yaml"
]

# Output directory in Google Drive
drive_output_base = "/content/drive/MyDrive/cherry_training/experiments"
data_root = "/content/cherry_classification/data"

# Ensure repo exists
if not os.path.exists("cherries"):
    print("Error: 'cherries' directory not found. Did the clone step succeed?")
else:
    for config_path in configs:
        config_name = Path(config_path).stem
        output_dir = f"{drive_output_base}/{config_name}"
        
        print(f"\n{'='*60}")
        print(f"STARTING EXPERIMENT: {config_name}")
        print(f"Config: {config_path}")
        print(f"Output: {output_dir}")
        print(f"{'='*60}\n")
        
        # Construct command
        # python cherries/training/scripts/train.py --config ...
        cmd = [
            "python", "cherries/training/scripts/train.py",
            "--config", config_path,
            "--output-dir", output_dir,
            "--data-root", data_root
        ]
        
        try:
            # Run with live output streaming
            process = subprocess.Popen(
                cmd, 
                stdout=subprocess.PIPE, 
                stderr=subprocess.STDOUT, 
                text=True, 
                bufsize=1
            )
            
            # Print output line by line
            for line in process.stdout:
                print(line, end='')
            
            process.wait()
            
            if process.returncode == 0:
                print(f"\nSUCCESS: Experiment {config_name} completed.")
            else:
                print(f"\nFAILURE: Experiment {config_name} failed with code {process.returncode}.")
                
        except Exception as e:
            print(f"\nERROR: Execution failed for {config_name}: {e}")
            
    print(f"\n{'='*60}")
    print("ALL EXPERIMENTS COMPLETED")
    print(f"{'='*60}")